## Clean reviews dataset
#### Friendly reminder: Please do not run this notebook since it will take about 30mins.
You can read the cleaned review dataset directly using the code below:
cleaned_review_df = pd.read_pickle('../data/data_cleaned/cleaned_review.zip')

### 0. Import reviews dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm
from langdetect import detect

pd.set_option('display.max_columns', 100)

RANDOM_STATE= 42

In [2]:
# load raw reviews dataset
review_df_raw = pd.read_csv('../data/data_raw/reviews.csv.gz', compression='gzip')

### 1. Clean raw reviews dataset

First, let's explore the review dataset

In [3]:
# review_df_raw.info()
# review_df_raw.isna().sum()
# review_df_raw.head()

#### Note: 

After we cleaned the reivews dataset, we **directly load the cleaned_review dataset** instead of rerun the code below for quick accessing. 

You can **use the code below to load the cleaned_review dataset directly**.
cleaned_review_df = pd.read_pickle('../data/data_cleaned/cleaned_review.zip')
print(cleaned_review_df.shape)
cleaned_review_df.head(2)


#### updates 3.9
1. dropped nan values in the comments
2. dropped canceled comment prototypes
3. dropped non-text comments (eg. --, .., /, emojis..)
4. dropped non-english comments (eg. fr, ro, zh-cn,..)

In [4]:
# updated 3.9
# dropped nan values in the comments

def clean_raw_review(df):
    # columns to remove
    review_col_to_remove = ['id', 'reviewer_name']
    cleaned_df = df.drop(review_col_to_remove,axis=1)
    
    # convert date to date type
    cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # convert listing_id datatype to object
    cleaned_df['listing_id'] = cleaned_df['listing_id'].astype('object')    
    
    # drop nans
    print("{:.2f}% of all comments are nans(Dropped).".format((cleaned_df['comments'].isna().sum())/cleaned_df.shape[0]*100))
    cleaned_df.dropna(subset=['comments'], axis = 0, inplace= True) 
    
    # drop canceled comment prototypes
    canceled_index = cleaned_df[cleaned_df['comments'].str.match('The host canceled this reservation')].index
    cleaned_df.drop(index = canceled_index,inplace= True)
    print("{:.2f}% of all comments are canceled comment prototypes(Dropped).".format(len(canceled_index)/cleaned_df.shape[0]*100))
    
    return cleaned_df

# review_df = clean_raw_review(review_df_raw)

0.01% of all comments are nans(Dropped).
0.11% of all comments are canceled comment prototypes(Dropped).


In [5]:
%%time
# detect languages # it takes about 18 mins to run
# (We'll use the pickled cleaned_review.zip to do further nlp analysis)
def detect_language(df):
    num_errors = 0
    non_text_index = []
    text_index = []
    languages = []

    # here it takes a long time to run
    for index, row in df.iterrows():
        try:
            language = detect(row['comments']) 
            languages.append(language)
            text_index.append(index)
        except:
            num_errors += 1 # LangDetectException: No features in text
            non_text_index.append(index)
#             print("This row throws an error:", row['comments'])    
    return non_text_index, text_index, languages

# non_text_index, text_index, languages = detect_language(review_df)

CPU times: user 27min 9s, sys: 18.2 s, total: 27min 27s
Wall time: 28min 6s


In [6]:
# len(non_text_index), len(text_index), len(languages)

(559, 383027, 383027)

In [7]:
# remove non-English comments
def remove_non_english_review(df):

    global non_text_index, text_index, languages
            
    # drop non-words comments (<0.15%)
    df.drop(non_text_index, axis=0, inplace=True)
    print("{:.2f}% of all comments do not contain words(Dropped).".format(len(non_text_index)/df.shape[0]*100))    
    # create a new col, language 
    df['language'] = languages    
    # drop non-english comments
    non_english_index = df[df['language'] != 'en'].index
    df.drop(index = non_english_index, inplace=True)
    print("{:.4f}% of all comments are not in English(Dropped).".format((len(non_english_index))/(df.shape[0]*100)))
    #drop language col
    review_df.drop('language', axis=1, inplace=True)
    
    return df

# review_df = remove_non_english_review(review_df)

0.15% of all comments do not contain words(Dropped).
0.0002% of all comments are not in English(Dropped).


In [8]:
review_df.reset_index(inplace=True, drop=True)
review_df.sample(2)

,listing_id,date,reviewer_id,comments
221650,24410427,2022-08-31,57400362,Very comfortable unit in a quiet location. The...
106001,10364350,2019-07-26,24713400,This stay met or exceeded all of my expectatio...


### 2. Save the cleaned reviews dataset

In [9]:
# save the cleaned data 
review_df.to_pickle('../data/data_cleaned/cleaned_review.zip')

In [10]:
# read the cleaned review dataset directly
cleaned_review_df = pd.read_pickle('../data/data_cleaned/cleaned_review.zip')
print(cleaned_review_df.shape)
cleaned_review_df.head(2)

(375173, 4)


,listing_id,date,reviewer_id,comments
0,6606,2009-07-17,18085,"The Urban Cottage is comfortable, beautiful, f..."
1,491958,2012-07-09,2595909,"Jay, Colleen, Mica & Gigi-dog were excellent h..."


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f2a50dc6-ff6a-45ff-9dbe-d7a35bd1e393' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>